In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import json

torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
class MultiQueryAttention(nn.Module):
    """
    Mult-query attention: multiple queries, but shared key and value.

    """

    def __init__(self, dim, num_heads=4, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5 #scales down attention scores so gradient remains stables 

        self.q_proj = nn.Linear(dim, dim) 

        self.k_proj
        